In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pinn-reproduction-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4650022%2F7914326%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240414%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240414T153026Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9c79f3149f0b06ce863480ddcf021570e160b051194fe0f48026437c11d332a2fde1a61297421cc497b500ae25544e5da287ed6146807e563ce6481a94f152a4dc64585f347ca5657a4f8607421765814c686dc7a32d92d6b6ac45baa4f7b4b3a39cfea8ed6c01573f7385a3311e06e1f1fc73e8154e90663c786af6752d7726a382f172658b6bf399e82f0c5a1a1f3e8c14ddd4e51c8995e56807269fc369e5ad2aa59566a226bb3add2cd4c75e6f3e2f8aa77288ba376aa99693eaa0e3a5b92b3eaecc8f5e7d27caf34fb2528bb1d9716c25c97266b6e44381295a135936cbe50d4c33fb4417d22d8b2c595c51611aa03bdc6db1a23a1506c3bed746e41c3c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import os

# Set the backend for DeepXDE
#os.environ['DDE_BACKEND'] = 'pytorch'
'''BE SURE TO INSTALL WURLITZER TOO, TENSORFLOW NEED IT APPARANTLY'''
#!pip install wurlitzer
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
#import torch
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib

#print(torch.__version__)

# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
''' Ensure this is uncommented to install DeepXDE if not already available'''
#!pip install deepxde

import deepxde as dde
# from torch.utils.data import TensorDataset, DataLoader
# from torch.utils.tensorboard import SummaryWriter
# from torchvision import datasets, transforms, models
from tqdm import tqdm

In [ ]:
#DeepXDE implementation of neural network
print(dde.__version__)
# np.random.seed(42)
# torch.random.seed()
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)
dde.backend.tf.random.set_random_seed(seed)

num_output = 2
PINN_DeepXDE = dde.nn.FNN([1] + [64] * 3 + [num_output], "tanh", "Glorot normal")
# test_input = torch.linspace(0,1, steps=101).view(-1, 1).float()
# input_shape  = test_input.shape

# testmodel2 = PINN_DeepXDE(test_input)

# output_shape2 = testmodel2.shape
# print('--===PENDULUM MODEL DeepXDE===--')
# print(f'The model has an input shape of {input_shape} [time vector] and an output shape of {output_shape2} [theta, torque].')

In [ ]:
# ==================================================================
# PYTORCH METHOD, DO NOT DELETE UNLESS TUTOR AGREES
# ==================================================================


# class PINN_torch(nn.Module):
#     def __init__(self, input_size=1, output_size=2):
#         super(PINN_torch, self).__init__()
#         self.node_num = 64
#         self.input_size = input_size
#         self.output_size = output_size
#         #Input layer
#         self.linear_input = nn.Linear(self.input_size, self.node_num)
#         self.tanh = nn.Tanh()
#         #Hidden layer
#         self.linear_hidden = nn.Linear(self.node_num,self.node_num)
#         self.linear_output = nn.Linear(self.node_num,self.output_size)

#     def forward(self, x):
#         #input
#         x = self.linear_input(x)
#         x = self.tanh(x)
#         #hidden
#         x = self.linear_hidden(x)
#         x = self.tanh(x)
#         x = self.linear_hidden(x)
#         x = self.tanh(x)
#         x = self.linear_hidden(x)
#         x = self.tanh(x)
#         #output
#         y = self.linear_output(x)
#         return y
# test_input = torch.linspace(0,1, steps=101).view(-1, 1).float()
# testmodel = PINN_torch()
#input_shape  = test_input.shape
# output_shape = testmodel(test_input).shape
# print('--===PENDULUM MODEL torch===--')
# print(f'The model has an input shape of {input_shape} [time vector] and an output shape of {output_shape} [theta, torque].')
# print(testmodel.parameters())
# print(testmodel(test_input).shape)

In [ ]:
# Set physical parameters, seed and hyper parameters
# Idk what hyper parameters we need
n_output = 2 # theta, torq_norm
num_domain = 1000 #sampling points in dataset WITHIN the domain, so NOT on boundaries.
num_boundary = 2 #We only have two boundaries because we're only working in the time domain. t_start and t_end, so we only need to sample on those two points.
n_adam = 10000 #originally was 5000, but manually tweaking values to see if the model can still learn eventhough it fucks up during its learning process
lr = 2e-2 # for Adam 2e-2 originally
loss_weights = [1., 10., 1., 1., 1.]


t_start = 0
t_end = 10.0
m = 1.
l = 1.
g = 9.8
torque_max = 1.5
torque_min = -1.5
goal = -1 #Goal is cos(th(t=10s)) = -1
target = goal


In [ ]:
# ==================================================================
# PYTORCH METHOD, DO NOT DELETE UNLESS TUTOR AGREES
# ==================================================================

# #setting up constraints and other costs with pytorch method
# def grad_fn(outputs, inputs):
#     grad = torch.autograd(outputs, inputs, grad_outputs = torch.ones_like(outputs), create_graph = True)
#     return grad

# def L_phys(model: torch.nn.Module):
#     ts = torch.linspace(t_min,t_max, steps = 101)
#     [th,torque] = model(ts)
#     torch.clamp(torque,min=torque_min,max=torque_max)
#     th_d = grad(th, ts)[0]
#     th_dd = grad(th_d,ts)[0]
#     pde = m*l**2*th_dd - (torque - m*g*l*torch.sin(th))
#     return torch.mean(pde**2) #loss function of physics informed part

In [ ]:
#Setting up constraints and other costs with deepxde method
def L_phys(time, NN_output):
    '''
    Based on the time and input u, calculate the time derivatives of theta and return the ode.
    This function is fed when creating dataset, as an argument to dde.data.PDE().

    Inputs:
    time: time vector
    NN_output: output of the neural network consisting of theta and torque

    Output:
    ode: ordinary differential equation

    ===============================================================================
    TODO: try applying tanh() on the torque variable, clamp is not differentiable
    ===============================================================================

    '''

    th = NN_output[:,0:1] #if I don't do 0:1 it gives out of bounds error
    torque = tf.tanh(NN_output[:,1:2]) #if I don't to 1:2 it gives out of ounds error
    #torch.clamp(torque,min=torque_min,max=torque_max)
    torque = torque*torque_max
    th_d = dde.grad.jacobian(th, time)
    th_dd= dde.grad.jacobian(th_d, time)
    ode = m * (l**2) * th_dd - (torque - m * g * l * tf.sin(th))

    return ode



time_domain = dde.geometry.TimeDomain(t_start, t_end) #specifying the time domain over which this problem will need to be solved
#Setting up the initial conditions with dde.icbc.IC(geom, func, on_initial, component=0).


# Creating your custom boundary condition class. Notice that this takes the same input as the dde.icbc.BC commands below.

# Couldn't really find a way to properly understand this. But from what I understand, the author needs this specific costum boundary class in order to properly describe what the boundary values are for t=tend.
# So in other words: What value should my neural network outputs have at t_final. As far as I could see, there isn't really a better way to do this than to create your own costum boundary function.

class Custom_BC(dde.icbc.BC):

    def __init__(self, geom, func, on_boundary, component=0):
        '''
        Initializer method of the Custom_BC class. Allows to create a custom boundary condtion of the clas Custom_BC.
        Is required for the definition of boundary conditions for th_final.


        Inputs:

        geom: The geom argument represents the geometry over which the problem is defined.
              In the context of differential equations, this usually refers to the spatial domain for spatial
              problems or the time domain for temporal problems.
              For initial conditions, this typically specifies the time at which the initial conditions
              are applied (often at the start of the simulation).

        func: The func argument is a function that specifies the values of the initial conditions.
              This function takes the coordinates in the geometry as input (for a time-dependent problem,
              this would be time t) and returns the values that the solution should have at those coordinates at the
              start of the simulation.

        on_initial:  The on_initial argument is a boolean function that determines which points in the geometry are
                     considered the initial points. This function takes coordinates as input and returns
                     True for coordinates that correspond to the initial condition and False otherwise.

        on_boundary: A function which returns True/False based on wether you are on the boundary.

        component: The component argument specifies which component of the system the initial condition applies to.
                   This is relevant for systems of equations where each equation might have different
                   initial conditions


        For more info on the creation of the boundary_conditions function (hard to understand and boring,
        do not read unless you are super interested in python):

        https://deepxde.readthedocs.io/en/latest/_modules/deepxde/icbc/boundary_conditions.html

        '''
        # The line below allows to inherit the initializer function of a regular dde.icbc.BC() object. In other words,
        # it creates parameters self.geom, self.on_boundary and self.component
        super().__init__(geom, on_boundary, component)

        # Create parameter function self.func which describes the boundary condition function
        # npfunc_range_autocache() is a specific type of function used to define the BC.
        self.func = dde.icbc.boundary_conditions.npfunc_range_autocache(dde.utils.return_tensor(func))



    def error(self, X, inputs, outputs, beg, end, aux_var=None):
        '''
        Calculate the error between the goal (target) value and actual values.

        Inputs:

        X:
        ?

        inputs:
        not used

        outputs:
        Output of the NN consisting of theta and torque.

        beg:
        Initial index.

        end:
        Last index.

        aux_var:
        ?

        Output:
        Error value betwen target and actual.
        '''

        values = self.func(X, beg, end, aux_var)
        th = outputs[:, 0:1]
        #goal = (th/np.pi)*tf.cos(th)
        goal = tf.cos(th)

        return goal[beg:end, self.component:self.component + 1] - values


def init_cond(t):
    '''
    Return 0 for any input "t". This is used to specify zero initial value.

    Input:
    t: variable to be set to 0

    Output:
    Array of zeros.
    '''
    return np.array([0.])


def final_cond(t):
    '''
    Create array of final, target values for any input t.

    Input:
    t: variable to be set to 0

    Output:
    Array of target values.
    '''
    return np.array([goal])


def initial(_, on_initial):
    '''
    ???
    IDK WHY, BUT CODE WILL GIVE ERRORS IF I DON'T DO IT LIKE THIS
    '''
    return on_initial


def min_value_for_boundary(t, on_boundary):
    '''
    Define the initial conditions. Required for Neuman boundary conditions.
    In principle, isclose() checks if the t[0] is the same as t_start returning logical (boolean) vector.
    It is then multiplied by on_boundary vector. This is nececary for numerical stability

    Inputs:
    t: time vector

    on_boundary: vector to be multiplied by the boolean vector created by isclose()

    Outputs:
    Vector of initial conditions.

    '''
    return on_boundary * np.isclose(t[0], t_start)

def max_value_for_boundary(t, on_boundary):
    '''
    Define what our maximum value boundary is for our time domain. Required for Neuman boundary conditions.

    Inputs:
    t: time vector

    on_boundary: vector to be multiplied by the boolean vector created by isclose()

    Outputs:
    Vector of maximum value bondary conditions.
    '''
    return on_boundary * np.isclose(t[0], t_end)

# Setting initial condition at t=0 for theta to 0.
th_0   = dde.icbc.IC(geom = time_domain, func = init_cond ,on_initial = initial, component = 0)
# Setting initial condition at t=0 for torque to 0
tau_0  = dde.icbc.IC(geom = time_domain, func = init_cond,on_initial = initial, component = 1)

th_d_0 = dde.icbc.NeumannBC(geom = time_domain, func = init_cond, on_boundary = min_value_for_boundary)
# SEE EXPLANATION FOR THIS BELOW
# Due to theta_dot not being in the Neural network output, we need to use a NeumanBC
# A neuman BC is a boundary condition for the derivative of a variable
# So setting geom = timedomain means that we're taking the time derivate, and func = init_cond sets the value/boundary to be 0.
# Finally, because we can't use dde.icbc.IC, we need to somehow tell this boundary condition, that the boundary is the initial condition (t=0).
# We do this by saying, boundary_on = initial_for_neuman_boundary



th_final = Custom_BC(geom = time_domain, func = final_cond, on_boundary = max_value_for_boundary) #boundary condition that tells system the angle should be pi at t=10s.

**Now that we've defined our boundary conditions and governing physics, we can start building/defining the dataset we feed to the PINN¶**


In [ ]:
#using deepxde.data.pde.PDE(geometry, pde, bcs, num_domain=0, num_boundary=0, train_distribution='Hammersley', anchors=None, exclusions=None, solution=None, num_test=None, auxiliary_var_function=None)
'''
geometry:                     Instance of Geometry.
pde:                          A global PDE or a list of PDEs. None if no global PDE.
bcs:                          A boundary condition or a list of boundary conditions. Use [] if no boundary condition.
num_domain (int):             The number of training points sampled inside the domain.
num_boundary (int):           The number of training points sampled on the boundary.
train_distribution (string):  The distribution to sample training points. One of the following:
                              “uniform” (equispaced grid), “pseudo” (pseudorandom), “LHS” (Latin hypercube sampling), “Halton” (Halton sequence), “Hammersley” (Hammersley sequence), or “Sobol” (Sobol sequence).
anchors:                      A Numpy array of training points, in addition to the num_domain and num_boundary sampled points.
exclusions:                   A Numpy array of points to be excluded for training.
solution:                     The reference solution.
num_test:                     The number of points sampled inside the domain for testing PDE loss.
                              The testing points for BCs/ICs are the same set of points used for training. If None, then the training points will be used for testing.
auxiliary_var_function:       A function that inputs train_x or test_x and outputs auxiliary variables.
'''

data = dde.data.PDE(geometry = time_domain, pde = L_phys, bcs = [th_0, tau_0, th_d_0, th_final],
                    num_domain = num_domain, num_boundary = num_boundary )#the rest is just kept as default settings.
'''see at which value it blows up (if any) and set an aditional constraint on it to not reach that value'''
print("Data generation done")

PINN_model = dde.Model(data, PINN_DeepXDE)
print("Model made")



#Training part, idk if this needs to be changed, I just copy pasted this part.

resampler = dde.callbacks.PDEPointResampler(period=100)

'''Here I try to save the best performing model parameters and then use that model instead of the model parameters corresponding to the last learning step'''



#dde.optimizers.config.set_LBFGS_options(ftol=np.nan, gtol=np.nan, maxiter=8000, maxfun=8000)
print("ADAM Training starting...")
PINN_model.compile("adam", lr=lr, loss_weights=loss_weights)
losshistory, train_state = PINN_model.train(display_every=10, iterations=n_adam, callbacks=[resampler])
print("ADAM Training Finished!")

print("L-BFGS Algorithm starting")
dde.optimizers.config.set_LBFGS_options(gtol=1e-10, maxiter=8000)
PINN_model.compile("L-BFGS",loss_weights=loss_weights)#, loss_weights=loss_weights)
losshistory, train_state = PINN_model.train(display_every=10)
print("L-BFGS Algorithm done")

# print("L-BFGS Training starting...")
# PINN_model.compile("L-BFGS", loss_weights=loss_weights)
# losshistory, train_state = PINN_model.train(display_every=1000)
# print("L-BFGS Training Finished!")



In [ ]:
'plotting the shit'
dde.saveplot(losshistory, train_state, issave=True, isplot=True)
PINN_model.save('Saved_model_Pendulum')


t = np.linspace(t_start, t_end, 101)
uu = PINN_model.predict(np.array([t]).T)
plt.figure()
plt.plot(t, uu[:, 0])
plt.title('angle over time')
plt.show()


plt.figure()
plt.plot(t, np.tanh(uu[:, 1]))
plt.title('normalized torque input (using tanh)')
plt.show()

plt.figure()
plt.plot(t, uu[:, 1])
plt.title('"actual" torque input')
plt.show()

total_loss_train = ([loss[0] for loss in losshistory.loss_train])
total_loss_test = ([loss[0] for loss in losshistory.loss_test])

bopus = np.sum(losshistory.loss_train, axis=1)
# Plot the total training loss
plt.figure#(figsize=(10, 6))
plt.yscale('log')
plt.plot(np.linspace(0,10000,1004),bopus, label='Total Train Loss', color='blue')
plt.xlabel('Iteration')
plt.ylabel('Total Loss')
plt.title('Total Loss Over Iterations')
plt.legend()
plt.savefig("Loss_breakdown.png")
plt.show()

# Plot the total test loss
plt.figure(figsize=(10, 6))
plt.plot(total_loss_test, label='Total Test Loss', color='red')
plt.xlabel('Iteration')
plt.ylabel('Total Loss')
plt.title('Total Test Loss Over Iterations')
plt.legend()
plt.show()

In [ ]:
'''plotting stuff'''
font = 12
#matplotlib.rcParams['axes.linewidth']=1.5
matplotlib.rcParams['axes.labelsize']=font
matplotlib.rcParams['axes.titlesize']=font
matplotlib.rcParams['xtick.labelsize']=font
matplotlib.rcParams['ytick.labelsize']=font
import matplotlib.pyplot as plt
from matplotlib.patches import Arc, RegularPolygon
from numpy import radians as rad

n_output = 2 # theta, torq_norm
tmin, tmax = 0.0, 10.0
torq_max = 1.5
m = 1.
l = 1.
g = 9.8
target = -1.

t_plot = [2., 5.2, 6.4, 8., 9.5]


def drawCirc(ax,radius,centX,centY,angle_,theta2_,color_='black',zorder=None,sign=1):
    #========Line
    arc = Arc([centX,centY],radius,radius,angle=angle_,
          #theta1=0,theta2=theta2_,capstyle='round',linestyle='-',lw=10,color=color_)
          theta1=0,theta2=theta2_,capstyle='round',linestyle='-',lw=2.5,color=color_,zorder=zorder)
    ax.add_patch(arc)


    #========Create the arrow head
    if sign >= 0:
        endX=centX+(radius/2)*np.cos(rad(theta2_+angle_)) #Do trig to determine end position
        endY=centY+(radius/2)*np.sin(rad(theta2_+angle_))
    else:
        endX=centX+(radius/2)*np.cos(rad(angle_)) #Do trig to determine end position
        endY=centY+(radius/2)*np.sin(rad(angle_))

    ax.add_patch(                    #Create triangle as arrow head
        RegularPolygon(
            (endX, endY),            # (x,y)
            3,                       # number of vertices
            radius/3,                # radius
            rad(angle_+theta2_),     # orientation
            color=color_,
            zorder=zorder
        )
    )
    #ax.set_xlim([centX-radius,centY+radius]) and ax.set_ylim([centY-radius,centY+radius])
    # Make sure you keep the axes scaled or else arrow will distort
t = np.linspace(tmin, tmax, 501)
uu = PINN_model.predict(np.array([t]).T)

fig, ax = plt.subplots(1, 1, figsize=(5, 1.5))
ax.plot(t, uu[:, 0], 'k', label='Angle')
ax.plot(t, torq_max * np.tanh(uu[:, 1]), 'b', label='Torque')

ax.axhline(np.pi, c='r', ls='--', label='Goal')
ax.axhline(-np.pi, c='r', ls='--')
ax.axhline(0, c='k', ls='--', lw=0.5, zorder=-1)
for tt in t_plot:
    ax.axvline(tt, c='darkgray', lw=0.5)
ax.set_xlim([tmin, tmax])
#ax.set_ylim([-np.pi - 0.2, np.pi + 0.2])

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
#plt.legend()
plt.tight_layout()
plt.savefig('pendulum_pinn.png')
#plt.show()

'''
# Check
dt = t[1] - t[0]

theta = np.zeros_like(t)
omega = np.zeros_like(t)
torque = np.tanh(uu[:, 1]) * torq_max
x = uu[:, 0]
x_t = np.append([0.], np.diff(x)) / dt
x_tt = np.append([0.], np.diff(x_t)) / dt
y = m * g * l * np.sin(x) + m * l * l * x_tt

for i in range(1, len(t)):
    omega_t = (torque[i] - m * g * l * np.sin(theta[i - 1])) / (m * l * l)
    omega[i] = omega[i - 1] + dt * omega_t
    theta[i] = theta[i - 1] + dt * omega[i] + 0.5 * dt ** 2 * omega_t

fig, ax = plt.subplots(1, 1, figsize=(5, 2))
ax.plot(t, uu[:, 0], 'k', label='Angle')
ax.plot(t, theta, 'k--', label='Angle')
ax.plot(t, torque, 'b', label='Torque')
ax.plot(t, y, 'b--', label='Torque')

ax.axhline(np.pi, c='r', ls='--', label='Goal')
ax.axhline(-np.pi, c='r', ls='--')
ax.axhline(0, c='k', ls='--', lw=0.5, zorder=-1)
ax.set_xlim([tmin, tmax])
#ax.set_ylim([-np.pi - 0.2, np.pi + 0.2])

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
#plt.legend()
plt.tight_layout()
plt.show()
'''

theta = uu[:, 0]
torque = torq_max * np.tanh(uu[:, 1])
di = 10

x = np.linspace(-np.pi, np.pi, 101)

fig, axs = plt.subplots(1, len(t_plot), sharey=True, figsize=(5, 1))
for it, tt in enumerate(t_plot):
    idx = np.abs(t - tt).argmin()
    xx, yy = theta[idx], torque[idx]
    xxm, yym = theta[idx - di], torque[idx - di]
    xxmm, yymm = theta[idx - 2 * di], torque[idx - 2 * di]
    axs[it].plot(l * np.cos(x), l * np.sin(x), 'k--', lw=0.5)
    axs[it].plot([0, l * np.sin(xx)], [0, -l * np.cos(xx)], 'gray', lw=3)
    axs[it].scatter([l * np.sin(xx)], [-l * np.cos(xx)], marker='o', color='k', s=30, zorder=10)
    axs[it].plot([0, l * np.sin(xxm)], [0, -l * np.cos(xxm)], 'gray', lw=3, alpha=0.5)
    axs[it].scatter([l * np.sin(xxm)], [-l * np.cos(xxm)], marker='o', color='k', s=30, zorder=10, alpha=0.5)
    axs[it].plot([0, l * np.sin(xxmm)], [0, -l * np.cos(xxmm)], 'gray', lw=3, alpha=0.2)
    axs[it].scatter([l * np.sin(xxmm)], [-l * np.cos(xxmm)], marker='o', color='k', s=30, zorder=10, alpha=0.2)

    if it == len(t_plot) - 1:
        axs[it].plot([0, 0], [0, l], 'r--', lw=1.5, zorder=25)

    angle = 120
    theta2 = 300
    #drawCirc(axs[it], 0.3 * np.abs(yy), 0, 0, angle, theta2, color_='b', zorder=20, sign=np.sign(yy))
    drawCirc(axs[it], 0.4 * np.abs(yy), 0, 0, angle, theta2, color_='b', zorder=20, sign=np.sign(yy))

    axs[it].set_xlim([-1.2, 1.2])
    axs[it].set_ylim([-1.2, 1.2])
    axs[it].axis('off')
    axs[it].set_aspect('equal', 'box')

#plt.tight_layout()
plt.savefig('snap.png')
#plt.show()


fig, axs = plt.subplots(2, 1, figsize=(1, 2))

x = np.linspace(-np.pi / 6, np.pi / 6)
axs[0].plot(l * np.sin(x), -l * np.cos(x), 'k--', lw=0.5)
axs[0].plot([0, l * np.sin(-np.pi / 8)], [0, -l * np.cos(-np.pi / 8)], 'gray', lw=3, alpha=0.3)
axs[0].scatter([l * np.sin(-np.pi / 8)], [-l * np.cos(-np.pi / 8)], marker='o', color='k', s=30, zorder=10, alpha=0.3)
axs[0].plot([0, l * np.sin(np.pi / 8)], [0, -l * np.cos(np.pi / 8)], 'gray', lw=3, alpha=1.)
axs[0].scatter([l * np.sin(np.pi / 8)], [-l * np.cos(np.pi / 8)], marker='o', color='k', s=30, zorder=10, alpha=1.)
drawCirc(axs[0], 0.4, 0, 0, angle, theta2, color_='b', zorder=20, sign=1)
axs[0].set_ylim([-1.4, 0.3])
axs[0].axis('off')
axs[0].set_aspect('equal', 'box')

x = np.linspace(-np.pi, np.pi, 101)
axs[1].plot(l * np.cos(x), l * np.sin(x), 'k--', lw=0.5)
axs[1].plot([0, 0], [0, l], 'gray', lw=3)
axs[1].scatter([0], [l], marker='o', color='k', s=30, zorder=10)
axs[1].set_ylim([-1.4, 1.4])
axs[1].axis('off')
axs[1].set_aspect('equal', 'box')

plt.savefig('intro.ong')
#plt.show()


#fig, axs = plt.subplots(1, 1, figsize=(3, 1))
fig, axs = plt.subplots(1, 1, figsize=(3, 1.5))

with open('loss.dat', 'r') as f:
    a = f.readlines()

step, phys_loss, const_loss, goal_loss = [], [], [], []
for line in a[1:]:
    step.append(float(line.split()[0]))
    phys_loss.append(float(line.split()[1]))
    const_loss.append(sum(list(map(float, line.split()[2:5]))))
    goal_loss.append(float(line.split()[5]))

axs.plot(step, phys_loss, 'b', label='Physics loss')
axs.plot(step, const_loss, 'r', label='Constraint loss')
axs.plot(step, goal_loss, 'g', label='Goal loss')
axs.plot(step, np.sum([phys_loss, const_loss, goal_loss], axis=0), 'k', lw=2, label='Sum')

axs.spines['right'].set_visible(False)
axs.spines['top'].set_visible(False)
axs.set_yscale('log')
axs.set_yticks([1e-8, 1e-4, 1e0])
#axs.set_yticks([1e-4, 1e-1, 1e2])

plt.tight_layout()
#plt.legend(fontsize=8, ncols=2)

plt.savefig('loss.png')
plt.show()

In [ ]:
import os

import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib
from tensorflow.python.framework import ops



import deepxde as dde
from tqdm import tqdm

In [ ]:
num_output = 2
n_output = 2
num_domain = 1000
num_boundary = 2
n_adam = 5000
lr = 2e-2
loss_weights = [1., 10., 1., 1., 1.]
t_start = 0
t_end = 10.0
m = 1.
l = 1.
g = 9.8
torque_max = 1.5
torque_min = -1.5
goal = -1 #Goal is cos(th(t=10s)) = -1
target = goal

#pre-allocating stuff for the average output and output per iteration.
avg_uu = np.zeros((501, 2))
uu = np.zeros((501,2,42))

for seed in range(30, 70):
  print("We're now at seed:",seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  dde.backend.tf.random.set_random_seed(seed)
  PINN_DeepXDE = dde.nn.FNN([1] + [64] * 3 + [num_output], "tanh", "Glorot normal")

  def L_phys(time, NN_output):
    th = NN_output[:,0:1] #if I don't do 0:1 it gives out of bounds error
    torque = tf.tanh(NN_output[:,1:2]) #if I don't to 1:2 it gives out of ounds error
    torque = torque*torque_max
    th_d = dde.grad.jacobian(th, time)
    th_dd= dde.grad.jacobian(th_d, time)
    ode = m * (l**2) * th_dd - (torque - m * g * l * tf.sin(th))

    return ode

  time_domain = dde.geometry.TimeDomain(t_start, t_end)
  #define boundary condition functions
  class Custom_BC(dde.icbc.BC):
    def __init__(self, geom, func, on_boundary, component=0):
        super().__init__(geom, on_boundary, component)
        self.func = dde.icbc.boundary_conditions.npfunc_range_autocache(dde.utils.return_tensor(func))
    def error(self, X, inputs, outputs, beg, end, aux_var=None):
        values = self.func(X, beg, end, aux_var)
        th = outputs[:, 0:1]
        goal = tf.cos(th)
        return goal[beg:end, self.component:self.component + 1] - values

  def init_cond(t):
      return np.array([0.])

  def final_cond(t):
      return np.array([goal])

  def initial(_, on_initial):
      return on_initial

  def min_value_for_boundary(t, on_boundary):
      return on_boundary * np.isclose(t[0], t_start)

  def max_value_for_boundary(t, on_boundary):
      return on_boundary * np.isclose(t[0], t_end)

  th_0   = dde.icbc.IC(geom = time_domain, func = init_cond ,on_initial = initial, component = 0)
  tau_0  = dde.icbc.IC(geom = time_domain, func = init_cond,on_initial = initial, component = 1)
  th_d_0 = dde.icbc.NeumannBC(geom = time_domain, func = init_cond, on_boundary = min_value_for_boundary)
  th_final = Custom_BC(geom = time_domain, func = final_cond, on_boundary = max_value_for_boundary)

  data = dde.data.PDE(geometry = time_domain, pde = L_phys, bcs = [th_0, tau_0, th_d_0, th_final],
                    num_domain = num_domain, num_boundary = num_boundary )
  #print("Data generation done")

  PINN_model = dde.Model(data, PINN_DeepXDE)
  #print("Model made")

  resampler = dde.callbacks.PDEPointResampler(period=100)

  #dde.optimizers.config.set_LBFGS_options(ftol=np.nan, gtol=np.nan, maxiter=8000, maxfun=8000)
  #print("ADAM Training starting...")
  PINN_model.compile("adam", lr=lr, loss_weights=loss_weights)
  losshistory, train_state = PINN_model.train(display_every=10, iterations=n_adam, callbacks=[resampler])
  #print("ADAM Training Finished!")

  #print("L-BFGS Algorithm starting")
  dde.optimizers.config.set_LBFGS_options(gtol=1e-10, maxiter=8000)
  PINN_model.compile("L-BFGS",loss_weights=loss_weights)#, loss_weights=loss_weights)
  losshistory, train_state = PINN_model.train(display_every=10)
  #print("L-BFGS Algorithm done")

  t = np.linspace(t_start, t_end, 501) #time axis
  uu[:,:,seed-30] = PINN_model.predict(np.array([t]).T)

  PINN_model = None
  losshistory = None
  train_state = None
  data = None
  th_0 = None
  tau_0 = None
  th_d_0 = None
  th_final = None
  time_domain = None
  PINN_DeepXDE = None

  del PINN_model, losshistory, train_state, data, th_0, tau_0, th_d_0, th_final, time_domain, PINN_DeepXDE

In [ ]:
#uu = uu[:,:,0:31]
uu1 = uu[:,0,:]
uu2 = uu[:,1,:]

avg_u = np.mean(uu, axis =2)
avg_u0 = avg_u[:,0]
avg_u1 = avg_u[:,1]

#std_uu = np.std(uu, axis=2)
std_uu0 = np.std(uu1, axis = 1)
std_uu1 = np.std(uu2, axis = 1)


t = np.linspace(0,10,len(avg_u))

plt.figure()
plt.plot(t, avg_u0, label='Average Value and deviation of torque (normalized)', color='blue')  # Plotting the average value
plt.fill_between(t, avg_u0 - std_uu0, avg_u0 + std_uu0, color='blue', alpha=0.2, label='Std Dev Range')  # Filling the std deviation range
plt.xlabel('Time')
plt.ylabel('torque (normalized)')
plt.title('Average and Std Dev Range for First Variable')
plt.legend()
plt.savefig("torque_std.png")
plt.show()

plt.figure()
plt.plot(t, avg_u1, label='Average Value and deviation of angle', color='blue')  # Plotting the average value
plt.fill_between(t, avg_u1 - std_uu1, avg_u0 + std_uu1, color='blue', alpha=0.2, label='Std Dev Range')  # Filling the std deviation range
plt.xlabel('Time')
plt.ylabel('angle')
plt.title('Average and Std Dev Range for First Variable')
plt.legend()
plt.savefig("angle_std.png")
plt.show()


In [ ]:
fig, ax = plt.subplots()

# Loop over the third dimension to plot each variation
for i in range(uu.shape[2]):
    ax.plot(uu[:, 0, i], linestyle='dotted', label=f'Iteration {i+1}')  # Plotting each iteration with dotted lines

# Adding thick dotted black lines at +pi and -pi on the y-axis
ax.axhline(y=np.pi, color='black', linestyle='dotted', linewidth=2, label='y = +π')
ax.axhline(y=-np.pi, color='black', linestyle='dotted', linewidth=2, label='y = -π')

# Adding labels and title
ax.set_xlabel('Time or index')
ax.set_ylabel('angle')
ax.set_title('Plot of 30 Iterations with Reference Lines at ±π')

# To avoid a cluttered legend, you might want to selectively show labels
# It can be helpful to disable the iteration labels in the legend if there are too many lines
handles, labels = ax.get_legend_handles_labels()
# Only show the last two entries (the +π and -π lines) in the legend
#ax.legend(handles[-2:], labels[-2:], loc='lower left')

# Show the plot
plt.savefig("all angles.png")
plt.show()

In [ ]:
fig, ax = plt.subplots()

# Loop over the third dimension to plot each variation
for i in range(uu.shape[2]):
    ax.plot(uu[:, 1, i], linestyle='dotted', label=f'Iteration {i+1}')  # Plotting each iteration with dotted lines

# Adding thick dotted black lines at +pi and -pi on the y-axis
#ax.axhline(y=np.pi, color='black', linestyle='dotted', linewidth=2, label='y = +π')
#ax.axhline(y=-np.pi, color='black', linestyle='dotted', linewidth=2, label='y = -π')

# Adding labels and title
ax.set_xlabel('Time or index')
ax.set_ylabel('torque input')
ax.set_title('Plot of 30 Iterations')

# To avoid a cluttered legend, you might want to selectively show labels
# It can be helpful to disable the iteration labels in the legend if there are too many lines
handles, labels = ax.get_legend_handles_labels()
# Only show the last two entries (the +π and -π lines) in the legend
#ax.legend(handles[-2:], labels[-2:], loc='lower left')

# Show the plot
plt.savefig("all torques.png")
plt.show()